In [43]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import (
    Agent, Runner, trace, function_tool, input_guardrail,
    GuardrailFunctionOutput, OpenAIChatCompletionsModel,
)
from typing import Dict
from pydantic import BaseModel

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

In [44]:
load_dotenv()

True

In [45]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins: {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins: {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins: {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins: {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins: sk-proj-
Google API Key exists and begins: AI
DeepSeek API Key exists and begins: sk-
Groq API Key exists and begins: gsk_


In [46]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [47]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [48]:
# Deepseek
deepseek_client = AsyncOpenAI(
    base_url=DEEPSEEK_BASE_URL,
    api_key=deepseek_api_key
)
deepseek_model = OpenAIChatCompletionsModel(
    model="deepseek-chat",
    openai_client=deepseek_client
)

# Gemini
gemini_client = AsyncOpenAI(
    base_url=GEMINI_BASE_URL,
    api_key=google_api_key
)
gemini_model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=gemini_client
)

# Groq
groq_client = AsyncOpenAI(
    base_url=GROQ_BASE_URL,
    api_key=groq_api_key
)
llama3_3_model = OpenAIChatCompletionsModel(
    model="llama-3.3-70b-versatile",
    openai_client=groq_client
)

In [49]:
sales_agent1 = Agent(
    name="Deepseek Sales Agent",
    instructions=instructions1,
    model=deepseek_model
)
sales_agent2 = Agent(
    name="Gemini Sales Agent",
    instructions=instructions2,
    model=gemini_model
)
sales_agent3 = Agent(
    name="Llama3.3 Sales Agent",
    instructions=instructions3,
    model=llama3_3_model
)


In [50]:
description = "Write a cold sales mail"

tool1 = sales_agent1.as_tool(
    tool_name="sale_agents1",
    tool_description=description
)
tool2 = sales_agent2.as_tool(
    tool_name="sale_agents2",
    tool_description=description
)
tool3 = sales_agent3.as_tool(
    tool_name="sale_agents3",
    tool_description=description
)

In [51]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email("anh.leduy04@hcmut.edu.vn")  # Change to your verified sender
    to_email = To("duyanhlucas302@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(f'response: {response}')
    return {"status": "success"}

In [52]:
# Subject writer and HTML converter Agents
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(
    name="Email subject writer",
    instructions=subject_instructions,
    model="gpt-4o-mini"
)
subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Write a subject for a cold sales email"
)

html_converter = Agent(
    name="HTML email body converter",
    instructions=html_instructions,
    model="gpt-4o-mini"
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a text email body to an HTML email body"
)

In [53]:
# Email Sender Agent
emailer_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name="Email Manager",
    instructions=emailer_instructions,
    tools=[subject_tool, html_tool, send_html_email],
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

In [ ]:
# Sales manager agent
# Note: Call multiple times can cause infinite loop
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=[tool1, tool2, tool3],
    handoffs=[emailer_agent],
    model="gpt-4o-mini"
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)
    print(result)


response: <python_http_client.client.Response object at 0x116991420>
response: <python_http_client.client.Response object at 0x116991630>
RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    The cold sales email has been successfully sent to the CEO with the subject line “Unlock Efficiency and Savings with AI Solutions – Let’s Chat!” If you need anything else or want to follow up, just let me know!
- 37 new item(s)
- 7 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### Guardrail

Guardrail: constraint around agent, designed to protect against unsafe input and against producing output which should not be shown for user.
- Guardrail can also be an agent
- Guardrail can only be applied at input (of the first agent) or output (of the last agent)

In [58]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput, # output an object follows this schema
    model="gpt-4o-mini"
)

In [59]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    # Run the guardrail_agent
    result = await Runner.run(
        guardrail_agent,
        message,
        context=ctx.context
    )
    is_name_in_message = result.final_output.is_name_in_message

    return GuardrailFunctionOutput(
        output_info={"found_name": result.final_output},
        tripwire_triggered=is_name_in_message # true if problem is detected (name in message)
        # trigger tripwire to cause this guardrail to fail
    )

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=[tool1, tool2, tool3],
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"
# Alice is a name in message -> should trigger the tripwire

with trace("Protected Automated SDR"):
    result = await Runner.run(
        careful_sales_manager,
        message
    )

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [63]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

MaxTurnsExceeded: Max turns (10) exceeded